In [2]:
# Обучение алгоритма R-learner
# Рассмотрим пример на данных Яндекс Такси. Что нужно сделать:
#   1. Построить предсказания out-of-fold для propensity score e(x) (вероятности попадания в группу воздействия).
#   2. Построить предсказания out-of-fold для таргета m(x) (без влияния воздействия).
#   3. Сформировать целевую переменную для R-learner.
#   4. Обучить финальную uplift-модель, минимизируя R-loss.
# 
# Задание 3
# Запустите код ниже. Рассчитайте метрики uplift AUC и Qini AUC для предсказаний алгоритма R-learner. 
#  Не забудьте применить метод squeeze(). Выведите значения метрик с точностью до двух знаков после запятой. 
# Используйте данные А/Б теста как и ранее.


from sklearn.model_selection import KFold
from xgboost import XGBRegressor, XGBClassifier
from sklift.metrics import uplift_auc_score, qini_auc_score
import numpy as np
import pandas as pd

data = pd.read_csv("ab_results.csv")

# Подготовка данных
features = data.drop(columns=['target','treatment']).columns
X = data[features].values
T = data.treatment.values  # 1 - группа воздействия, 0 - контрольная группа
y = data.target.values  # целевая переменная

# Создание фолдов для кросс-валидации
n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Получение предсказаний вне фолдов для e(x) (propensity score) и m(x) (базовый исход)
e_x = np.zeros(len(X))
m_x = np.zeros(len(X))

for train_idx, val_idx in kf.split(X):
    # Модель для e(x): вероятность назначения воздействия
    model_e = XGBClassifier(
        n_estimators=200,
        learning_rate=0.02,
        max_depth=6,
        random_state=42,
        verbosity=0
    )
    model_e.fit(X[train_idx], T[train_idx])
    e_x[val_idx] = model_e.predict_proba(X[val_idx])[:, 1]
    
    # Модель для m(x): предсказание исхода
    model_m = XGBRegressor(
        n_estimators=200,
        learning_rate=0.02,
        max_depth=6,
        random_state=42,
        verbosity=0
    )
    model_m.fit(X[train_idx], y[train_idx])
    m_x[val_idx] = model_m.predict(X[val_idx])

# Расчет целевой переменной для R-learner
r_target = y - m_x
r_treat = T - e_x

# Обучение финальной модели tau(x) (uplift-модель)
# Используем XGBoost-регрессию с весами sample_weight = r_treat**2
mask = np.abs(r_treat) > 1e-1
X_tau = X[mask]
r_target_tau = r_target[mask]
r_treat_tau = r_treat[mask]

tau_model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=6,
    random_state=42,
    verbosity=0
)
tau_model.fit(X_tau, r_target_tau, sample_weight=(r_treat_tau**2))

# Получение предсказаний uplift
uplift_scores = tau_model.predict(X)

# Расчет метрик
uplift_auc = uplift_auc_score(y, uplift_scores.squeeze(), T)
qini_auc = qini_auc_score(y, uplift_scores.squeeze(), T)

print(f"Uplift AUC: {uplift_auc:.2f}")
print(f"Qini AUC: {qini_auc:.2f}")


Uplift AUC: 0.07
Qini AUC: 0.09


In [3]:
# Задание 5
# Пора обучать R-learner с помощью библиотеки causalml. Ваша задача — определить переменные метода fit(), где:
# X — матрица признаков в обучающей выборке,
# treatment — факт воздействия в обучающей выборке,
# y — целевая переменная в обучающей выборке,
# p — оценки propensity score для обучающей выборки.
# Также следует прогнозы алгоритма на тестовой выборке без учёта столбца treatment.
# Мы предварительно разделили выборку на обучающую и тестовую:
# X_train — матрица признаков обучающей выборки,
# T — факт воздействия в обучающей выборке (бинарная переменная),
# y_train — целевая переменная в обучающей выборке (бинарная переменная).

import numpy as np
import pandas as pd
from causalml.inference.meta import BaseRClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklift.metrics import uplift_auc_score, qini_auc_score
from sklearn.model_selection import train_test_split


data = pd.read_csv("ab_results.csv")

X = data.drop(['target'], axis=1)  
y = data['target']  

# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                    stratify=data[['target', 'treatment']],
                                    random_state=42)

features = data.drop(columns=['target','treatment']).columns
T = X_train.treatment.values  # 1 - treatment, 0 - control
X_train = X_train[features]
y_train = y_train.values  # целевая переменная
T_test = X_test.treatment.values


# инициализируем R-learner с моделями XGBoost
# outcome_learner — модель для предсказания исхода без учёта воздействия
# effect_learner — модель для оценки эффекта воздействия
# propensity_learner — модель для оценки вероятности получения воздействия
r_learner = BaseRClassifier(
    outcome_learner=XGBClassifier(
        n_estimators=200,
        learning_rate=0.02,
        max_depth=6,
        random_state=42,
        verbosity=0
    ),
    effect_learner=XGBRegressor(
        n_estimators=200,
        learning_rate=0.01,
        max_depth=6,
        random_state=42,
        verbosity=0
    ),
    propensity_learner=XGBClassifier(
        n_estimators=200,
        learning_rate=0.02,
        max_depth=6,
        random_state=42,
        verbosity=0
    )
)

# инициализируем массив для propensity score
e_x = np.zeros(len(X_train))
T_test = X_test.treatment.values

# обучаем модель для оценки propensity score
model_e = XGBClassifier(n_estimators=100, 
                        learning_rate=0.02, 
                        max_depth=3, 
                        random_state=42, 
                        verbosity=0)
model_e.fit(X_train, T)
e_x = model_e.predict_proba(X_train)[:, 1]

# обучаем модель R-learner
r_learner.fit(
    X=X_train,
    treatment=T,
    y=y_train,
    p=e_x,
    verbose=True
)

# получаем оценки uplift-эффекта для тестовой выборки
uplift_scores = r_learner.predict(X_test.drop(columns=['treatment']).values)

# рассчитываем метрики качества модели
uplift_auc = uplift_auc_score(y_test, uplift_scores.squeeze(), T_test)
qini_auc = qini_auc_score(y_test, uplift_scores.squeeze(), T_test)

# выводим результаты
print(f"R-learner Uplift AUC: {uplift_auc:.2f}")
print(f"R-learner Qini AUC: {qini_auc:.2f}")


R-learner Uplift AUC: 0.16
R-learner Qini AUC: 0.20


In [4]:
# Задание 7
# Обучите propensity_model (модель вероятности назначения воздействия) с помощью XGBClassifier из библиотеки xgboost. 
# Также сделайте прогноз propensity score для обучающей и тестовой выборок с помощью метода predict_proba.
# В качестве T_train и T_test возьмите столбец воздействия (treatment) для обучающей и тестовой выборки соответственно. 
# Переопределите X_train и X_test без столбцов conversion и treatment. Используйте данные Яндекс.Плюса как и ранее.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

data = pd.read_csv("yandex_plus.csv")

# Разделим данные на признаки и целевую переменную
X = data.drop(['conversion'], axis=1)  
y = data['conversion']  

treatment_mapping = {
    'control': 0,  # 0 соответствует контрольной группе
    'treatment1': 1  # 1 соответствует группе воздействия
}

X['treatment'] = X['treatment'].map(treatment_mapping)

# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                    stratify=data[['conversion', 'treatment']],
                                    random_state=42)

features = data.drop(columns=['conversion','treatment']).columns
T_train = X_train.treatment.values  # 1 - treatment, 0 - control
X_train = X_train[features]
y_train = y_train.values  # целевая переменная
T_test = X_test.treatment.values
X_test = X_test[features]


model_e = XGBClassifier(n_estimators=300, 
                        learning_rate=0.1, 
                        max_depth=4, 
                        random_state=42, 
                        verbosity=0)

model_e.fit(X_train, T_train)
# получаем propensity score для train и test
p_train = model_e.predict_proba(X_train.values)[:, 1]
p_test = model_e.predict_proba(X_test.values)[:, 1]

roc_auc_test = roc_auc_score(y_test, p_test)

print("ROC_AUC на тестовой выборке равен", round(roc_auc_test, 2))

ROC_AUC на тестовой выборке равен 0.56


In [5]:
# Задание 9
# Обучите R-learner для бизнес-кейса Яндекс Плюс с помощью библиотеки causalml. Для этого:
#   Импортируйте необходимые классы:
#       BaseRClassifier из causalml.inference.meta, 
#       XGBClassifier и XGBRegressor из xgboost.
#   Создайте R-learner:
#       используйте BaseRClassifier,
#       укажите outcome_learner,
#       укажите effect_learner,
#       установите control_name=0.
#   Обучите модель:
#       используйте метод fit(),
#       передайте обучающую выборку, данные о воздействии, таргет обучающей выборки, 
#           propensity score из предыдущего задания для обучающей выборки (p_train).
# 
# Не забудьте про random_state=42 для воспроизводимости экспериментов.

N_ESTIMATORS = 50 
LEARNING_RATE = 0.005 
MAX_DEPTH = 6


r_learner = BaseRClassifier(
    outcome_learner=XGBClassifier(
        n_estimators=N_ESTIMATORS,
        learning_rate=LEARNING_RATE,
        max_depth=MAX_DEPTH,
        random_state=42,
        verbosity=0
    ),
    effect_learner=XGBRegressor(
        n_estimators=N_ESTIMATORS,
        learning_rate=LEARNING_RATE,
        max_depth=MAX_DEPTH,
        random_state=42,
        verbosity=0
    ),
    propensity_learner=XGBClassifier(
        n_estimators=N_ESTIMATORS,
        learning_rate=LEARNING_RATE,
        max_depth=MAX_DEPTH,
        random_state=42,
        verbosity=0
    )
)


# Обучите R-learner на обучающей выборке
r_learner.fit(
    X=X_train,
    treatment=T_train,
    y=y_train,
    verbose=True,
)

# Получаем оценки uplift эффекта для тестовой выборки
uplift_scores = r_learner.predict(X_test[features].values)

# Рассчитываем метрики качества модели
uplift_auc = uplift_auc_score(y_test, uplift_scores.squeeze(), T_test)
qini_auc = qini_auc_score(y_test, uplift_scores.squeeze(), T_test)

# Выводим результаты
print(f"R-learner Uplift AUC: {uplift_auc:.2f}")
print(f"R-learner Qini AUC: {qini_auc:.2f}")

R-learner Uplift AUC: 0.42
R-learner Qini AUC: 0.37
